In [2]:
from transformers import GPT2TokenizerFast, GPTNeoForCausalLM, GPTNeoConfig, AutoTokenizer, AlbertTokenizer, GPT2Tokenizer, RobertaTokenizer, RobertaTokenizerFast
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset, load_from_disk
from transformers import RobertaForCausalLM

from tokenizers import Tokenizer, pre_tokenizers, decoders, AddedToken, normalizers, trainers
from tokenizers.normalizers import BertNormalizer
from tokenizers.models import BPE, Unigram, WordLevel, WordPiece
from tokenizers.trainers import BpeTrainer, WordLevelTrainer, \
                                WordPieceTrainer, UnigramTrainer

from tokenizers.implementations import SentencePieceUnigramTokenizer

from tokenizers.processors import RobertaProcessing, TemplateProcessing
from tqdm import tqdm

import json
import wandb

In [35]:
def bits_per_token(token, encoding='utf-8'):
    # Calculate the number of bits for the token based on the encoding
    return len(token)

def average_bits_per_token(vocabulary):
    total_bits = 0
    for token in vocabulary:
        total_bits += bits_per_token(token)
    average_bits = total_bits / len(vocabulary)
    return average_bits

def clean_special_characters_from_token(token):
    return token.replace("Ġ", "").replace("▁", "").replace("#", "")

def clean_special_tokens(vocab):
    vocab.remove("<pad>")
    vocab.remove("<s>")
    vocab.remove("</s>")
    vocab.remove("<unk>")
    vocab.remove("<mask>")

In [36]:
bpe_tokenizers = {tokenizer_name: RobertaTokenizer.from_pretrained(f'./tokenizers/bpe/{tokenizer_name}/', model_max_length=512) for tokenizer_name in ["1k-bpe", "3k-bpe", "6k-bpe", "10k-bpe", "15k-bpe", "20k-bpe"]}

wp_tokenizers = {tokenizer_name: RobertaTokenizerFast.from_pretrained(f'./tokenizers/wordpiece/{tokenizer_name}/', model_max_length=512) for tokenizer_name in ["1k-wp", "3k-wp", "6k-wp", "10k-wp", "15k-wp", "20k-wp"]}

sp_tokenizers = {tokenizer_name: AlbertTokenizer.from_pretrained(f'./tokenizers/sentence-piece/{tokenizer_name}/', model_max_length=512) for tokenizer_name in ["1k-sp", "3k-sp", "6k-sp", "10k-sp", "15k-sp", "20k-sp"]}


In [37]:
def calculate_vocabs_avg_bits_per_token(vocabs):
    results = {}
    
    for model_name, tok in vocabs.items():
        vocab_list = [token for token, _ in tok.get_vocab().items()]
        clean_special_tokens(vocab_list)
        cleaned_vocab = [clean_special_characters_from_token(token) for token in vocab_list]
        # print(cleaned_vocab)
        avg_characters_per_token = average_bits_per_token(cleaned_vocab)
        results[model_name] = {"avg_char_per_token": avg_characters_per_token}
    return results

In [38]:
print(calculate_vocabs_avg_bits_per_token(wp_tokenizers))

{'1k-wp': {'avg_char_per_token': 2.02713567839196}, '3k-wp': {'avg_char_per_token': 3.3512520868113524}, '6k-wp': {'avg_char_per_token': 4.074061718098416}, '10k-wp': {'avg_char_per_token': 4.527863931965983}, '15k-wp': {'avg_char_per_token': 4.83407802600867}, '20k-wp': {'avg_char_per_token': 5.0324081020255065}}


In [39]:
def add_to_wandb(result):
    ''' Log results to wandb. For this you need to map the run name to its id 
        in the table, by downloading name, id columns from wandb. 
        In hindsight, it's probably easier to save this under the model to avoid duplicates. 
    '''
    run_id = ""
    with open('./run_ids.json') as json_file:
        run_ids = json.load(json_file)
        model = result['model']
        run_id = run_ids[model]

    # resume the wandb run and log the result
    wandb.init(
        entity='tiny-transformers', project='tokenizers', id=run_id, resume='must'
    )
    print(result)
    wandb.log(result)
    wandb.finish()

In [40]:
bpe_tok_results = calculate_vocabs_avg_bits_per_token(bpe_tokenizers)
wp_tok_results = calculate_vocabs_avg_bits_per_token(wp_tokenizers)
sp_tok_results = calculate_vocabs_avg_bits_per_token(sp_tokenizers)

all_results = bpe_tok_results | wp_tok_results | sp_tok_results
model_name_results = {}

run_id = ""
with open('./run_ids.json') as json_file:
    run_ids = json.load(json_file)
    model_names = [model_name for model_name, _ in run_ids.items()]
    for tok_name, results in all_results.items():
        model_name_matches = [name for name in model_names if tok_name in name]
        for mn in model_name_matches:
            model_name_results[mn] = results
        
# print(model_name_results)
results_to_upload = []

for model_name, result in model_name_results.items():
    results_to_upload.append({"model": model_name, **result})
    
for result in results_to_upload:
    add_to_wandb(result)

{'model': 'BERT-1k-bpe-9.0M-2L-4H-780C-1024I', 'avg_char_per_token': 2.5346733668341708}


avg_char_per_token,▁
avg_bits_per_token,21.30653
avg_char_per_token,2.53467
base_avg,0.5615
blimp_avg,0.57399
boolq,0.61826
cola,-0.02077
eval/loss,0.46485
eval/runtime,31.5492
eval/samples_per_second,697.006
eval/steps_per_second,43.583


{'model': 'GPT-1k-bpe-9.0M-2L-4H-780C-1024I', 'avg_char_per_token': 2.5346733668341708}


avg_char_per_token,▁
avg_bits_per_token,21.30653
avg_char_per_token,2.53467
base_avg,0.54083
blimp_avg,0.53707
boolq,0.61134
cola,0
eval/loss,1.29524
eval/runtime,29.0791
eval/samples_per_second,756.214
eval/steps_per_second,47.285


{'model': 'BERT-3k-bpe-9.0M-2L-4H-700C-1024I', 'avg_char_per_token': 3.5869782971619366}


...

avg_char_per_token,▁
avg_bits_per_token,44.52257
avg_char_per_token,5.55659
base_avg,0.62164
blimp_avg,0.61234
boolq,0.64039
cola,0.04729
eval/loss,1.67756
eval/runtime,39.4255
eval/samples_per_second,557.76
eval/steps_per_second,34.876


{'model': 'BERT-20k-sp-9.0M-2L-4H-348C-1024I', 'avg_char_per_token': 5.734233558389597}


avg_char_per_token,▁
avg_bits_per_token,45.93828
avg_char_per_token,5.73423
base_avg,0.62126
blimp_avg,0.60963
boolq,0.62517
cola,0
eval/loss,1.0846
eval/runtime,42.5246
eval/samples_per_second,517.113
eval/steps_per_second,32.334


{'model': 'GPT-20k-sp-9.0M-2L-4H-348C-1024I', 'avg_char_per_token': 5.734233558389597}


avg_char_per_token,▁
avg_bits_per_token,45.93828
avg_char_per_token,5.73423
base_avg,0.61221
blimp_avg,0.58578
boolq,0.64454
cola,0.04055
eval/loss,1.71803
eval/runtime,41.6309
eval/samples_per_second,528.214
eval/steps_per_second,33.028
